# Day 09. Exercise 01
# Gridsearch

## 0. Imports

In [37]:
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from tqdm.auto import tqdm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

/home/danka/bootcampDS/DS_Bootcamp.Day09/src/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Preprocessing

1. Read the file [`day-of-week-not-scaled.csv`](https://drive.google.com/file/d/1AlGvsJDSzPT_70caausx8bFuupIEZkfh/view?usp=sharing). It is similar to the one from the previous exercise, but this time we did not scale continuous features (we are not going to use logreg anymore). Don't forget to enrich the table with the 'dayofweek' column from the previous day's .csv-file.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [6]:
df=pd.read_csv("../data/day-of-week-not-scaled.csv", sep=',')
df['dayofweek']=pd.read_csv("../data/dayofweek.csv", usecols=['dayofweek'])
df.head()

,numTrials,hour,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,uid_user_15,uid_user_16,uid_user_17,uid_user_18,uid_user_19,uid_user_2,uid_user_20,uid_user_21,uid_user_22,uid_user_23,uid_user_24,uid_user_25,uid_user_26,uid_user_27,uid_user_28,uid_user_29,uid_user_3,uid_user_30,uid_user_31,uid_user_4,uid_user_6,uid_user_7,uid_user_8,labname_code_rvw,labname_lab02,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1,dayofweek
0,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
2,3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
3,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
4,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4


In [10]:
X=df.drop(columns='dayofweek')
y=df['dayofweek']

In [11]:
X_train, X_test, y_train, y_test=train_test_split(X, y, random_state=21, test_size=0.2, stratify=y)

## 2. SVM gridsearch

1. Using `GridSearchCV` try different parameters of kernel (`linear`, `rbf`, `sigmoid`), C (`0.01`, `0.1`, `1`, `1.5`, `5`, `10`), gamma (`scale`, `auto`), class_weight (`balanced`, `None`) use `random_state=21` and `probability=True` and get the best combination of them in terms of accuracy.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`. Check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [14]:
model_svc=SVC()
parametrs_grid={
    'kernel':['linear', 'rbf', 'sigmoid'],
    'C': [0.01, 0.1, 1, 1.5, 5, 10],
    'gamma':['scale', 'auto'],
    'random_state':[21],
    'probability': [True],
    'class_weight':['balanced', None]
}
grid=GridSearchCV(estimator=model_svc, param_grid=parametrs_grid, cv=5, scoring='accuracy',n_jobs=-1)
grid.fit(X_train, y_train)

,estimator,SVC()
,param_grid,"{'C': [0.01, 0.1, ...], 'class_weight': ['balanced', None], 'gamma': ['scale', 'auto'], 'kernel': ['linear', 'rbf', ...], ...}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,C,10


In [15]:
grid.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_C', 'param_class_weight', 'param_gamma', 'param_kernel', 'param_probability', 'param_random_state', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [17]:
result=pd.DataFrame(grid.cv_results_)
result=result[['rank_test_score', 'params', 'mean_test_score','std_test_score']]
result=result.sort_values('rank_test_score').reset_index(drop=True)
result.head()



,rank_test_score,params,mean_test_score,std_test_score
0,1,"{'C': 10, 'class_weight': None, 'gamma': 'auto...",0.876109,0.018419
1,2,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.863500,0.010870
2,3,"{'C': 5, 'class_weight': None, 'gamma': 'auto'...",0.816018,0.008116
3,4,"{'C': 5, 'class_weight': 'balanced', 'gamma': ...",0.808608,0.021007
4,5,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.721052,0.034438


## 3. Decision tree

1. Using `GridSearchCV` try different parameters of `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use `random_state=21`.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [24]:
model_tree=DecisionTreeClassifier()
params_grid_tree={
    'max_depth':range(1,50),
    'class_weight':['balanced', None],
    'criterion': ['entropy', 'gini'],
    'random_state':[21]

}
grid_tree=GridSearchCV(estimator=model_tree, param_grid=params_grid_tree, cv=5, scoring='accuracy', n_jobs=16)
grid_tree.fit(X_train, y_train)

,estimator,DecisionTreeClassifier()
,param_grid,"{'class_weight': ['balanced', None], 'criterion': ['entropy', 'gini'], 'max_depth': range(1, 50), 'random_state': [21]}"
,scoring,'accuracy'
,n_jobs,16
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'gini'


In [25]:
grid_tree.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_class_weight', 'param_criterion', 'param_max_depth', 'param_random_state', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [26]:
pd.set_option('display.max_colwidth', None)
result_tree=pd.DataFrame(grid_tree.cv_results_)
result_tree=result_tree[['rank_test_score', 'params', 'mean_test_score', 'std_test_score']]
result_tree=result_tree.sort_values('rank_test_score').reset_index(drop=True)
result_tree

,rank_test_score,params,mean_test_score,std_test_score
0,1,"{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 22, 'random_state': 21}",0.873121,0.023998
1,2,"{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}",0.873121,0.026300
2,3,"{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 32, 'random_state': 21}",0.873116,0.023911
3,3,"{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 33, 'random_state': 21}",0.873116,0.023911
4,3,"{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 48, 'random_state': 21}",0.873116,0.023911
...,...,...,...,...
191,192,"{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 3, 'random_state': 21}",0.373906,0.044064
192,193,"{'class_weight': None, 'criterion': 'entropy', 'max_depth': 1, 'random_state': 21}",0.355330,0.009338
193,193,"{'class_weight': None, 'criterion': 'gini', 'max_depth': 1, 'random_state': 21}",0.355330,0.009338
194,195,"{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 1, 'random_state': 21}",0.286358,0.028376


## 4. Random forest

1. Using `GridSearchCV` try different parameters of `n_estimators` (`5`, `10`, `50`, `100`), `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use random_state=21.
2. Create a dataframe from the results of the gridsearch and sort it ascendengly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [27]:
model_forest=RandomForestClassifier()
params_grid_forest={
    'n_estimators' :[5, 10, 50, 100],
    'max_depth' : range(1, 50),
    'class_weight': ['balanced', None],
    'criterion' : ['entropy', 'gini'],
    'random_state' : [21]
}
grid_forest=GridSearchCV(estimator=model_forest, param_grid=params_grid_forest,cv=5, scoring='accuracy',n_jobs=16)
grid_forest.fit(X_train, y_train)

,estimator,RandomForestClassifier()
,param_grid,"{'class_weight': ['balanced', None], 'criterion': ['entropy', 'gini'], 'max_depth': range(1, 50), 'n_estimators': [5, 10, ...], ...}"
,scoring,'accuracy'
,n_jobs,16
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,50


In [29]:
result_forest=pd.DataFrame(grid_forest.cv_results_)
result_forest=result_forest[['rank_test_score', 'params', 'mean_test_score', 'std_test_score']]
result_forest=result_forest.sort_values('rank_test_score').reset_index(drop=True)
result_forest

,rank_test_score,params,mean_test_score,std_test_score
0,1,"{'class_weight': None, 'criterion': 'gini', 'max_depth': 28, 'n_estimators': 50, 'random_state': 21}",0.904290,0.010961
1,2,"{'class_weight': None, 'criterion': 'gini', 'max_depth': 31, 'n_estimators': 100, 'random_state': 21}",0.903547,0.014380
2,3,"{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 30, 'n_estimators': 50, 'random_state': 21}",0.902817,0.013554
3,4,"{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 34, 'n_estimators': 50, 'random_state': 21}",0.902809,0.013010
4,5,"{'class_weight': None, 'criterion': 'gini', 'max_depth': 37, 'n_estimators': 100, 'random_state': 21}",0.902806,0.010460
...,...,...,...,...
779,780,"{'class_weight': None, 'criterion': 'entropy', 'max_depth': 1, 'n_estimators': 5, 'random_state': 21}",0.353832,0.016467
780,781,"{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 2, 'n_estimators': 5, 'random_state': 21}",0.353110,0.021165
781,782,"{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'n_estimators': 5, 'random_state': 21}",0.346419,0.029749
782,783,"{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 1, 'n_estimators': 5, 'random_state': 21}",0.283390,0.011062


## 5. Progress bar

Gridsearch can be a quite long process and you may find yourself wondering when it will end.
1. Create a manual gridsearch for the same parameters values of random forest iterating through the list of the possible values and calculating `cross_val_score` for each combination. Try to increase `n_jobs`. The value `cv` for `cross_val_score` is 5.
2. Track the progress using the library `tqdm.notebook`.
3. Create a dataframe from the results of the gridsearch with the columns corresponding to the names of the parameters and `mean_accuracy` and `std_accuracy`.
4. Sort it descendingly by the `mean_accuracy`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [38]:
res_forest_manual=[]

for params in tqdm(ParameterGrid(params_grid_forest), desc='GridSearch_Manual'):
    model_forest=RandomForestClassifier(**params)
    score=cross_val_score(model_forest, X_train, y_train, cv=5, n_jobs=-1)
    res_forest_manual.append({
        'params' : params,
        'mean_accuracy' : score.mean(),
        'std_accuracy' : score.std()
    })





















































































































































































































































































































































































































































































































































































Exception ignored in: <function tqdm.__del__ at 0x7e642c563060>
Traceback (most recent call last):
  File "/home/danka/bootcampDS/DS_Bootcamp.Day09/src/venv/lib/python3.12/site-packages/tqdm/std.py", line 1148, in __del__
  File "/home/danka/bootcampDS/DS_Bootcamp.Day09/src/venv/lib/python3.12/site-packages/tqdm/notebook.py", line 279, in close
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <fun

In [40]:
result_manual=pd.DataFrame(res_forest_manual).sort_values('mean_accuracy', ascending=False).reset_index(drop=True)
result_manual

,params,mean_accuracy,std_accuracy
0,"{'class_weight': None, 'criterion': 'gini', 'max_depth': 28, 'n_estimators': 50, 'random_state': 21}",0.904290,0.010961
1,"{'class_weight': None, 'criterion': 'gini', 'max_depth': 31, 'n_estimators': 100, 'random_state': 21}",0.903547,0.014380
2,"{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 30, 'n_estimators': 50, 'random_state': 21}",0.902817,0.013554
3,"{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 34, 'n_estimators': 50, 'random_state': 21}",0.902809,0.013010
4,"{'class_weight': None, 'criterion': 'gini', 'max_depth': 40, 'n_estimators': 100, 'random_state': 21}",0.902806,0.010460
...,...,...,...
779,"{'class_weight': None, 'criterion': 'entropy', 'max_depth': 1, 'n_estimators': 5, 'random_state': 21}",0.353832,0.016467
780,"{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 2, 'n_estimators': 5, 'random_state': 21}",0.353110,0.021165
781,"{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'n_estimators': 5, 'random_state': 21}",0.346419,0.029749
782,"{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 1, 'n_estimators': 5, 'random_state': 21}",0.283390,0.011062


## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.

In [41]:
model_forest_best=RandomForestClassifier(class_weight=None, criterion='gini', max_depth=28, n_estimators=50, random_state=21)
model_forest_best.fit(X_train, y_train)
predict=model_forest_best.predict(X_test)
accuracy_score(y_test, predict)

0.9289940828402367